### Exporting keys

In [29]:
import configparser
import os

In [30]:
config_file = 'keys.cfg'

In [164]:
config = configparser.ConfigParser()
config.read(config_file)

['keys.cfg']

In [167]:
os.environ['SPOTIPY_CLIENT_ID'] = config['SPOTIFY']['CLIENT_ID']
os.environ['SPOTIPY_CLIENT_SECRET'] = config['SPOTIFY']['CLIENT_SECRET']
os.environ['SPOTIPY_REDIRECT_URI'] = 'https://5e318e279fe5.ngrok.io'

# NOTE: this is a hacky workaround since this is just an application for private use. Not a best practise
os.environ['OAUTH_CODE'] = config['SPOTIFY']['OAUTH_CODE']

### Spotify searching

#### Workaround for getting an OAuth 2.0 access and refresh token

* Write a simple server application that listens on a port
* Use `ngrok` to tunnel requests to that port
* Set `ngrok` URL as the redirect URI on Spotify API Dashboard
* Use the code returned to get access and refresh tokens

In [260]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials, SpotifyOauthError

class SpotifySearch:
    def __init__(self, creds=None):
        self.spotipy = spotipy.Spotify(auth_manager=SpotifyClientCredentials())
        if creds:
            self.spotipy = spotipy.Spotify(auth_manager=creds)
    
    def search(self, search_query):
        """
        Search through Spotify search API and return results
        """
        return self.spotipy.search(search_query)

    def this_user_id(self):
        return self.spotipy.me()['id']    
        
    def get_first_track(self, search_query):
        results = self.search(search_query)
        res = {}
        if len(results) > 0:
            try:
                first = results['tracks']['items'][0]
                res['url'] = first['external_urls']['spotify']
                res['api_url'] = first['href']
                res['uri'] = first['uri']
                res['id'] = first['id']
            except KeyError as e:
                print(f'[error] URLs/URIs not found for {search_query}')
        return res

    def get_playlist(self, playlist_name):
        playlists = self.spotipy.current_user_playlists()['items']
        username = self.spotipy.me()['display_name']
        
        playlist_store = {}
        for playlist in playlists:
            if playlist['name'] == playlist_name and playlist['owner']['display_name'] == username:
                playlist_store = playlist
                break
        
        if len(playlist_store) > 0:
            return {
                'id': playlist_store['id'],
                'name': playlist_store['name'],
                'uri': playlist_store['uri'],
            }

        return {}
    
    def add_to_playlist(self, playlist, track):
        self.spotipy.user_playlist_add_tracks(
            user=self.this_user_id(),
            playlist_id=playlist['id'],
            tracks=track['uri']
        )


In [216]:
spotify = SpotifySearch()
spotify.get_first_track('bts stay gold')

{'url': 'https://open.spotify.com/track/0UwQTbq7ZtBJ7v530mpaxP',
 'api_url': 'https://api.spotify.com/v1/tracks/0UwQTbq7ZtBJ7v530mpaxP',
 'uri': 'spotify:track:0UwQTbq7ZtBJ7v530mpaxP'}

In [ ]:
scopes = [
    'playlist-modify-public', 'playlist-modify-private',
    'playlist-read-collaborative', 'playlist-read-private',
    'user-library-read', 'user-library-modify'
]
scope = ' '.join(scopes)
scope

In [188]:
from spotipy.oauth2 import SpotifyOAuth
oauth_creds = SpotifyOAuth(scope=scope, username='aadithpm')

In [261]:
spotify = SpotifySearch(creds=oauth_creds)

In [262]:
spotify.get_playlist('Running')

{'id': '7FqCCwSi3cWdbTUVpBOmIP',
 'name': 'Running',
 'uri': 'spotify:playlist:7FqCCwSi3cWdbTUVpBOmIP'}

In [263]:
track = spotify.get_first_track('bts mic drop')
track

{'url': 'https://open.spotify.com/track/75scDPqGs75FotglJSoOI2',
 'api_url': 'https://api.spotify.com/v1/tracks/75scDPqGs75FotglJSoOI2',
 'uri': 'spotify:track:75scDPqGs75FotglJSoOI2',
 'id': '75scDPqGs75FotglJSoOI2'}

In [264]:
playlist = spotify.get_playlist('Korean')
playlist

{'id': '2J3QAUvsQISpVfl03hlNLL',
 'name': 'Korean',
 'uri': 'spotify:playlist:2J3QAUvsQISpVfl03hlNLL'}

In [265]:
spotify.add_to_playlist(playlist, track)

HTTP Error for POST to https://api.spotify.com/v1/users/s0vjmwx4s6oi5szb5u3byak4j/playlists/2J3QAUvsQISpVfl03hlNLL/tracks returned 400 due to Invalid track uri: spotify:track:s


SpotifyException: http status: 400, code:-1 - https://api.spotify.com/v1/users/s0vjmwx4s6oi5szb5u3byak4j/playlists/2J3QAUvsQISpVfl03hlNLL/tracks:
 Invalid track uri: spotify:track:s